# OpenSNP data formatting and analysis
#### Adam Klie<br>05/20/2020<br>CSE284 Project
Notebook to look at genotypes and phenotypes in OpenSNP

In [65]:
import pandas as pd
import os
import numpy as np
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from scipy.stats import zscore

## 1. Retrieve user IDs with eye color phenotype data from OpenSNP
 - Input: 
     - `phenotypes_202004220659.csv` -- phenotype data for downloaded openSNP genotype files
 - Output: 
     - `openSNP_initial_phenotypes.tab` -- a tabular version of initial set of users with phenotype data (638)
     - `openSNP_initial_userid.list` -- list of user IDs (one per line) for those users with eye color phenotype data
 - Notes:
     - Only looking at individuals with the following eye colors: 'Brown', 'brown', 'Blue', 'blue', 'Green', 'green'
     - In the following columns: "Eye color", "Eye Color", "Eye pigmentation "
     - 638 such individuals come out of this

In [66]:
# Read in OpenSNP phenotypes
phenotypes = pd.read_csv('phenotypes_202004220659.csv', sep=';')[['user_id', 'genotype_filename', 'date_of_birth', 
                                                                  'chrom_sex', 'Eye color', 'Eye Color', 
                                                                  'Eye pigmentation ']]

In [67]:
# Remove non-reporters
phenotypes = phenotypes[(phenotypes["Eye color"] != '-') | (phenotypes["Eye Color"] != '-') | (phenotypes["Eye pigmentation "] != '-')]

In [68]:
# Uncomment to look at useful phenotype values
#phenotypes["Eye color"].value_counts()
#phenotypes["Eye Color"].value_counts()
#phenotypes["Eye pigmentation "].value_counts()

In [69]:
# Only using these phenotype labels
valid_phenotypes = ['Brown', 'brown', 'Blue', 'blue', 'Green', 'green']

In [70]:
# Only take those samples that have valid phenotypes in one of these three columns
phenotypes = phenotypes[(phenotypes["Eye color"].isin(valid_phenotypes)) | 
                        (phenotypes["Eye Color"].isin(valid_phenotypes)) | 
                        (phenotypes["Eye pigmentation "].isin(valid_phenotypes))]

In [71]:
# Collapes to a single column
def collapseEyeColors(x, values):
    if x["Eye color"] not in values:
        if x["Eye Color"] in values:
            return x["Eye Color"].lower()
        elif x["Eye pigmentation "] in values:
            return x["Eye pigmentation "].lower()
    else:
        return x["Eye color"].lower()

In [72]:
# Use function to collapse eye color metadata to single columns
phenotypes["eye_color"] = phenotypes.apply(lambda x: collapseEyeColors(x, valid_phenotypes), 1)

In [73]:
# Remove duplicate individuals
phenotypes = phenotypes.drop_duplicates(subset="user_id")

In [81]:
# Only take Ancestry and 23andMe data to simplify
phenotypes = phenotypes[phenotypes["genotype_filename"].str.contains('ancestry') | phenotypes["genotype_filename"].str.contains('23andme')]

In [82]:
print(phenotypes["eye_color"].value_counts())
print(phenotypes["eye_color"].value_counts().sum())

brown    333
blue     208
green     97
Name: eye_color, dtype: int64
638


In [83]:
phenotypes.to_csv('openSNP_initial_phenotypes.tab', sep='\t', index=False)

In [84]:
# Save this id set as a list
ids = set(phenotypes["user_id"].values)
with open("openSNP_initial_userid.list", 'w') as filehandle:
    filehandle.writelines("%s\n" % id for id in ids)

## 2. Extracting initial genotypes using user_ids with eye_color phenotypes and filtering for high coverage SNPS
**Run `extractGenotypes.py` to generate a tsv of rsids vs genotypes (`openSNP_initial_genotypes.tab`)**<br>
 - Input: 
     - `openSNP_initial_genotypes.tab` -- genotype data for downloaded openSNP genotype files
 - Output: 
     - `openSNP_filtered_genotypes.tab` -- a tabular version of genotypes filtered
     - `openSNP_filtered_rsids.list` -- list of rsids to pass to 1000Genomes to find intersection with
 - Notes:
     - Filter this dataframe to bring the number of SNPs down (currently at least 80% of samples have rsid)

In [78]:
# AncestryDNA example data
#!head -20 ~/cse284/datasets/openSNP/genotypes/user1001_file496_yearofbirth_unknown_sex_unknown.ancestry.txt
#!wc -l ~/cse284/datasets/openSNP/genotypes/user1001_file496_yearofbirth_unknown_sex_unknown.ancestry.txt

# 23andMe example data
#!head -20 ~/cse284/datasets/openSNP/genotypes/user1002_file497_yearofbirth_unknown_sex_unknown.23andme.txt
#!wc -l ~/cse284/datasets/openSNP/genotypes/user1002_file497_yearofbirth_unknown_sex_unknown.23andme.txt

# FTDNA example data
#!head -20 ~/cse284/datasets/openSNP/genotypes/user1018_file3018_yearofbirth_unknown_sex_XY.ftdna-illumina.txt

In [3]:
# Read in genotypes
initial_genotypes = pd.read_csv('openSNP_initial_genotypes.tab', sep='\t')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (161,183,242,376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
num_SNPs = len(initial_genotypes)
num_samples = len(initial_genotypes.loc[:, "6":"6131"].columns)

In [5]:
print("After extracting genotypes we have {} SNP x {} sample matrix".format(num_SNPs, num_samples))

After extracting genotypes we have 1269707 SNP x 593 sample matrix


In [ ]:
# Make sure we have iris SNPs captured
IRISPLEX = os.path.join(os.environ["HOME"], "project/datasets/", "irisplex.bed")
iris_header = ["chr", "pos1", "pos2", "id", "minor_allele", "b1", "b2"]
iris = pd.read_csv(IRISPLEX, sep='\t', header=None, names=iris_header)

In [ ]:
iris_snps = iris["id"].values
iris_gts = initial_genotypes[initial_genotypes["rsid"].isin(iris_snps)]
min(iris_gts.notna().apply(np.mean, 1))

In [ ]:
# Filter for SNPs that appear in at least 80% of samples (> than min iris SNP fraction)
common_snp_genotypes = genotypes[genotypes.loc[:,'6':].notna().apply(np.mean, 1) > 0.80]

In [ ]:
# Write this filtered set to a smaller dataframe
common_snp_genotypes.to_csv('openSNP_filtered_genotypes.tab', sep='\t', index=False)

In [ ]:
# Save this list of SNP rsids for oneK_genotpyes.sh
final_rsids = common_snp_genotypes["rsid"].values.tolist()
with open("openSNP_filtered_rsids.list", 'w') as filehandle:
    filehandle.writelines("%s\n" % id for id in final_rsids)

In [ ]:
print("After filtering, %d SNPs and %d samples left" % (len(final_rsids), len(user_id_list)))

## 4. Get final genotypes from filtered file and output of oneK genomes
 - Input: 
     - `openSNP_filtered_genotypes.tab` -- genotype data for downloaded openSNP genotype files
     - `../oneKGenomes/oneK_rsids.tab` -- see how to build this in ../oneKGenomes
 - Output: 
     - `openSNP_filtered_genotypes.tab` -- a tabular version of genotypes filtered
     - `openSNP_filtered_rsids.list` -- list of rsids to pass to 1000Genomes to find intersection with
 - Notes:
     - Comes from final SNP list from 1K genomes (run oneK_genotypes.sh to get oneK_rsids.tab)

In [85]:
# Read in filtered genotypes
filtered_genotypes = pd.read_csv('openSNP_filtered_genotypes.tab', sep='\t')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (376) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [86]:
# Read in the final SNP the final SNP set
oneK_ref = pd.read_csv('../oneKGenomes/oneK_rsids.tab', sep='\t')

In [87]:
# Replace -- with NaN
filtered_genotypes = filtered_genotypes.replace('--', np.nan)

In [88]:
# Keep only SNPs found in 1000Genomes
gt_char = filtered_genotypes.merge(oneK_ref, left_on="rsid", right_on="ID")

In [89]:
gt_char.head()

,rsid,chromosome,position,6,8,2056,4106,10,11,13,...,6117,2023,2024,6120,6124,4080,8178,6131,ID,REF
0,rs3131972,1,742584,AG,AG,AG,GG,AG,GG,AG,...,GG,GG,GG,GG,AG,AG,AA,GG,rs3131972,A
1,rs4970383,1,828418,CC,CC,AA,AC,AC,AC,CC,...,CC,CC,AC,CC,AC,CC,NaN,CC,rs4970383,C
2,rs4475691,1,836671,CC,CC,CC,CT,CT,CT,CC,...,CC,CC,CT,CC,CT,CC,CC,CC,rs4475691,C
3,rs7537756,1,844113,AA,AA,AA,AG,AG,AG,AA,...,AA,AA,AG,AA,AG,AA,AA,AA,rs7537756,A
4,rs13302982,1,851671,GG,GG,GG,GG,GG,GG,GG,...,GG,GG,GG,GG,GG,GG,AA,GG,rs13302982,A


In [90]:
# Function to convert genotype from nucleotides to numbers 0 (homozygous reference), 1(heteroozygous), 2 (homozygous minor allele)
# Alos fixes any SNPs with -- to NaN
def get_gt(x):
    for col in x.index[3:-2]:       
        if isinstance(x[col], str):
            ref_allele = x["REF"]
            gt = 2 - x[col].upper().count(ref_allele)
            x[col] = gt
    return x

In [91]:
# WARNING: Getting the numbers takes about an hour and half
gt_nums = gt_char.progress_apply(get_gt, axis=1)

In [94]:
# Save a final SNP vs indiviudal table with other SNP info in it
gt_nums = gt_nums.drop("ID", axis=1)
gt_nums.to_csv("openSNP_final_genotypes.tab", sep='\t', index=False)

In [95]:
#Function to z-score SNPs
def z_score(df): 
    return (df-df.mean())/df.std()

In [96]:
# Z-score SNPs
z_scored_snps = gt_nums.set_index("rsid").loc[:, "6":"6131"].progress_apply(z_score, axis=1)

In [118]:
test = z_scored_snps.T.loc[phenotypes[phenotypes["user_id"].isin(test.index)]["user_id"].astype(str).tolist()]

In [121]:
test = test.replace(np.nan, 0)

In [123]:
test.to_csv("test_set.csv", index=True)

In [124]:
test_ids = test.index

In [125]:
# Final user id list
with open("openSNP_final_userid.list", 'w') as filehandle:
    filehandle.writelines("%s\n" % id for id in test_ids)

## 5. Get final phenotypes from...

In [127]:
with open("openSNP_final_userid.list", 'r') as filename:
    final_user_ids = [line.rstrip() for line in filename.readlines()]
phenos = pd.read_csv('openSNP_initial_phenotypes.tab', sep='\t')

In [128]:
# Get the phenotypes
final_phenotypes = phenos[phenos["user_id"].isin(final_user_ids)]
final_phenotypes[["user_id", "eye_color"]].to_csv('openSNP_final_phenotypes.tab', sep='\t', index=False)

In [129]:
final_phenotypes["eye_color"].value_counts()

brown    312
blue     194
green     84
Name: eye_color, dtype: int64

In [130]:
final_phenotypes = final_phenotypes.set_index("user_id")

In [131]:
final_phenotypes.head()

,genotype_filename,date_of_birth,chrom_sex,Eye color,Eye Color,Eye pigmentation,eye_color
user_id,,,,,,,
4440,4440.ancestry.3043,1954,XX,Brown,-,-,brown
4441,4441.23andme.3044,rather not say,rather not say,Blue,-,-,blue
4106,4106.23andme.2779,rather not say,rather not say,Blue,-,-,blue
4171,4171.23andme.2830,rather not say,rather not say,Brown,-,-,brown
3973,3973.23andme.2669,1992,XY,Brown,-,-,brown


In [132]:
# Use numeric labels for phenotypes
label_mapping = {"brown":0, "blue":1, "green":2}
def get_label(x):
    return label_mapping[x["eye_color"]]

In [133]:
final_phenotypes["label"] = final_phenotypes.apply(get_label, 1)

In [134]:
final_phenotypes["label"].to_csv('test_labels.csv', index=True)

#### Test with iris

In [178]:
IRISPLEX = os.path.join(os.environ["HOME"], "project/datasets/", "irisplex.bed")
iris_header = ["chr", "pos1", "pos2", "id", "minor_allele", "b1", "b2"]
iris = pd.read_csv(IRISPLEX, sep='\t', header=None, names=iris_header)
iris.head()

,chr,pos1,pos2,id,minor_allele,b1,b2
0,15,28365618,28365619,rs12913832,A,-4.81,-1.79
1,15,28230318,28230319,rs1800407,T,1.40,0.87
2,14,92773663,92773664,rs12896399,G,-0.58,-0.03
3,5,33951693,33951694,rs16891982,C,-1.30,-0.50
4,11,89011046,89011047,rs1393350,A,0.47,0.27


In [179]:
iris_snps = iris["id"].values.tolist()
iris_snps

['rs12913832',
 'rs1800407',
 'rs12896399',
 'rs16891982',
 'rs1393350',
 'rs12203592']

In [140]:
#gt = gt_nums
gts = pd.read_csv("openSNP_final_genotypes.tab", sep='\t')

In [182]:
iris_genotypes = gts[gts["rsid"].isin(iris_snps)]
iris_genotypes

,rsid,chromosome,position,6,8,2056,4106,10,11,13,...,2018,6117,2023,2024,6120,6124,4080,8178,6131,REF
82354,rs16891982,5,33987450,2,2.0,2.0,2.0,1.0,1.0,2.0,...,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,NaN,C
95430,rs12203592,6,341321,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,C
181970,rs1393350,11,88650694,0,0.0,0.0,2.0,1.0,0.0,1.0,...,2.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,G
218381,rs12896399,14,91843416,0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,NaN,1.0,2.0,0.0,1.0,2.0,0.0,NaN,G
220888,rs1800407,15,25903913,0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,C
220908,rs12913832,15,26039213,1,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,A


In [183]:
iris_merged = iris_genotypes.merge(oneK_ref, left_on="rsid", right_on="ID")
iris_merged

,rsid,chromosome,position,6,8,2056,4106,10,11,13,...,2023,2024,6120,6124,4080,8178,6131,REF_x,ID,REF_y
0,rs16891982,5,33987450,2,2.0,2.0,2.0,1.0,1.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,NaN,C,rs16891982,C
1,rs12203592,6,341321,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,C,rs12203592,C
2,rs1393350,11,88650694,0,0.0,0.0,2.0,1.0,0.0,1.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,G,rs1393350,G
3,rs12896399,14,91843416,0,0.0,1.0,1.0,0.0,0.0,1.0,...,1.0,2.0,0.0,1.0,2.0,0.0,NaN,G,rs12896399,G
4,rs1800407,15,25903913,0,1.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,C,rs1800407,C
5,rs12913832,15,26039213,1,0.0,2.0,2.0,0.0,0.0,0.0,...,2.0,2.0,1.0,1.0,2.0,1.0,1.0,A,rs12913832,A


In [184]:
final_iris = iris_genotypes.merge(iris, left_on="rsid", right_on="id")

In [203]:
final_iris

,rsid,chromosome,position,6,8,2056,4106,10,11,13,...,8178,6131,REF,chr,pos1,pos2,id,minor_allele,b1,b2
0,rs16891982,5,33987450,2,2.0,2.0,2.0,1.0,1.0,2.0,...,2.0,NaN,C,5,33951693,33951694,rs16891982,C,-1.30,-0.50
1,rs12203592,6,341321,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,C,6,396321,396322,rs12203592,T,0.70,0.73
2,rs1393350,11,88650694,0,0.0,0.0,2.0,1.0,0.0,1.0,...,0.0,1.0,G,11,89011046,89011047,rs1393350,A,0.47,0.27
3,rs12896399,14,91843416,0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,NaN,G,14,92773663,92773664,rs12896399,G,-0.58,-0.03
4,rs1800407,15,25903913,0,1.0,0.0,2.0,0.0,0.0,0.0,...,1.0,1.0,C,15,28230318,28230319,rs1800407,T,1.40,0.87
5,rs12913832,15,26039213,1,0.0,2.0,2.0,0.0,0.0,0.0,...,1.0,1.0,A,15,28365618,28365619,rs12913832,A,-4.81,-1.79


In [204]:
def fix_genotypes(x):
    if x["rsid"] in ['rs12896399', 'rs12913832', 'rs16891982']:
        for col in x.index[3:-8]:
            x[col] = 2 - x[col]
    return x

In [205]:
final_iris_fixed = final_iris.apply(fix_genotypes, 1)
final_iris_fixed

,rsid,chromosome,position,6,8,2056,4106,10,11,13,...,8178,6131,REF,chr,pos1,pos2,id,minor_allele,b1,b2
0,rs16891982,5,33987450,0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,NaN,C,5,33951693,33951694,rs16891982,C,-1.30,-0.50
1,rs12203592,6,341321,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,C,6,396321,396322,rs12203592,T,0.70,0.73
2,rs1393350,11,88650694,0,0.0,0.0,2.0,1.0,0.0,1.0,...,0.0,1.0,G,11,89011046,89011047,rs1393350,A,0.47,0.27
3,rs12896399,14,91843416,2,2.0,1.0,1.0,2.0,2.0,1.0,...,2.0,NaN,G,14,92773663,92773664,rs12896399,G,-0.58,-0.03
4,rs1800407,15,25903913,0,1.0,0.0,2.0,0.0,0.0,0.0,...,1.0,1.0,C,15,28230318,28230319,rs1800407,T,1.40,0.87
5,rs12913832,15,26039213,1,2.0,0.0,0.0,2.0,2.0,2.0,...,1.0,1.0,A,15,28365618,28365619,rs12913832,A,-4.81,-1.79


In [206]:
# Code to compute probabilities for all individuals
import pandas as pd
import numpy as np
import os

# Functions for model probability calculations
def p_blue(m1_sum, m2_sum):
    return np.exp(m1_sum)/(1+np.exp(m1_sum)+np.exp(m2_sum))

def p_other(m1_sum, m2_sum):
    return np.exp(m2_sum)/(1+np.exp(m1_sum)+np.exp(m2_sum))

# Perform predictions for each individual based on genotype and parameters
a1 = 3.94
a2 = 0.65

predictions = {}
for col in final_iris_fixed.columns[3:-10]:
    pred1 = np.dot(final_iris_fixed[col], final_iris_fixed["b1"]) + a1
    pred2 = np.dot(final_iris_fixed[col], final_iris_fixed["b2"]) + a2
    blue = p_blue(pred1, pred2)
    other = p_other(pred1, pred2)
    brown = 1 - blue - other
    predictions[col] = [pred1, pred2, blue, other, brown]

predict = pd.DataFrame.from_dict(predictions, orient='index', columns=["pred1", "pred2", "blue", 
                                                                       "other", "brown"])

In [207]:
def get_color_pred(x):
    if x["blue"] >= x["brown"] and x["blue"] >= x["other"]:
        return "blue"
    elif x["brown"] >= x["other"]:
        return "brown"
    else:
        return "other"

In [208]:
predict["predicted_eye_color"] = predict.apply(get_color_pred, 1)

In [209]:
predict["predicted_eye_color"].value_counts()

brown    242
blue     231
other    118
Name: predicted_eye_color, dtype: int64

In [210]:
predictions = predict.reset_index()
predictions['user_id'] = predictions['index'].astype(int)
predictions.head()

,index,pred1,pred2,blue,other,brown,predicted_eye_color,user_id
0,6,-1.33,-0.47,0.139974,0.330780,0.529246,brown,6
1,8,-5.44,-2.12,0.003859,0.106754,0.889386,brown,8
2,2056,3.36,0.62,0.909665,0.058737,0.031597,blue,2056
3,4106,7.10,2.90,0.984426,0.014762,0.000812,blue,4106
4,10,-7.67,-3.22,0.000448,0.038403,0.961149,brown,10


In [221]:
result_nona["predicted_eye_color"].value_counts()

brown    242
blue     230
other      8
Name: predicted_eye_color, dtype: int64

In [214]:
result = phenotypes.merge(predictions, on="user_id")
result_nona = result.dropna()

In [216]:
result_nona["pred_correct"] = result_nona["eye_color"].str.lower().values == result_nona["predicted_eye_color"].str.lower().values

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [220]:
result_nona.groupby("eye_color").mean()

,user_id,pred1,pred2,blue,other,brown,pred_correct
eye_color,,,,,,,
blue,3898.638037,4.075276,1.188957,0.914405,0.058675,0.026920,0.98773
brown,3764.271255,-3.344615,-1.643117,0.119612,0.167862,0.712526,0.94332
green,4333.642857,2.657286,0.639571,0.764856,0.124405,0.110740,0.00000


In [215]:
np.mean(result_nona["eye_color"].str.lower().values == result_nona["predicted_eye_color"].str.lower().values)

0.8208333333333333